# Disaster Tweet Analysis: EDA, Cleaning, BERT  
Auto‑generated notebook skeleton based on your original report  
Author: **Mohamed Niyaz**  
Created: 2025‑07‑31

**How to use:**  
1. Make sure `train.csv` and `test.csv` from the Kaggle “Real or Not? Disaster Tweets” competition are in the same folder as this notebook (or change the paths in `load_data`).  
2. Run the notebook top‑to‑bottom.  
3. Heavy BERT training is commented out—uncomment those lines if you have GPU/V100 or Colab Pro.  


In [2]:
# ==============================
# 1. Library Imports & Seed
# ==============================
import gc, re, string, random, operator
from collections import defaultdict
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import os
os.environ["TRANSFORMERS_NO_KERAS_TENSOR"] = "1"

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import STOPWORDS

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

import tensorflow as tf
# import tensorflow_hub as hub
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from transformers import BertTokenizer, TFBertModel

SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)


c:\Users\niyaz\anaconda3\envs\nlp-tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ==============================
# 2. Data Loading
# ==============================
def load_data(train_path='./Data/train.csv', test_path='./Data/test.csv'):
    """Load the Kaggle disaster tweets dataset."""
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)
    print(f'Training set shape: {df_train.shape}')
    print(f'Test set shape:     {df_test.shape}')
    return df_train, df_test

df_train, df_test = load_data()


Training set shape: (7613, 5)
Test set shape:     (3263, 4)


In [4]:
# ==============================
# 3. Text Cleaning Utilities
# ==============================
URL_PATTERN = re.compile(r'https?://\S+|www\.\S+')
HTML_PATTERN = re.compile(r'<.*?>')
MENTION_PATTERN = re.compile(r'@[A-Za-z0-9_]+')
HASHTAG_PATTERN = re.compile(r'#')
PUNCTUATION_TABLE = str.maketrans('', '', string.punctuation)

def clean_text(text: str) -> str:
    text = URL_PATTERN.sub('', text)
    text = HTML_PATTERN.sub('', text)
    text = MENTION_PATTERN.sub('', text)
    text = HASHTAG_PATTERN.sub('', text)
    text = text.translate(PUNCTUATION_TABLE)
    return text.lower().strip()

for df in (df_train, df_test):
    df['text_cleaned'] = df['text'].fillna('').apply(clean_text)


In [5]:
# ==============================
# 4. Feature Engineering
# ==============================
def add_meta_features(df):
    df['word_count'] = df['text'].str.split().apply(len)
    df['unique_word_count'] = df['text'].str.split().apply(lambda x: len(set(x)))
    df['stop_word_count'] = df['text'].str.lower().str.split().apply(lambda x: len([w for w in x if w in STOPWORDS]))
    df['url_count'] = df['text'].str.contains('http|https').astype(int)
    df['mean_word_length'] = df['text'].str.split().apply(lambda x: np.mean([len(w) for w in x]) if x else 0)
    df['char_count'] = df['text'].str.len()
    df['punctuation_count'] = df['text'].apply(lambda x: len([c for c in x if c in string.punctuation]))
    df['hashtag_count'] = df['text'].str.count('#')
    df['mention_count'] = df['text'].str.count('@')
    return df

df_train = add_meta_features(df_train)
df_test  = add_meta_features(df_test)


In [6]:
# ==============================
# 5. BERT Classifier (HF version)
# ==============================
class BertTweetClassifier:
    def __init__(self, max_seq_length=160, lr=1e-5, epochs=2, batch_size=16, folds=3):
        self.max_seq_length = max_seq_length
        self.lr, self.epochs, self.batch_size, self.folds = lr, epochs, batch_size, folds
        # HF tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def encode(self, texts):
        enc = self.tokenizer(
            texts.tolist(),
            truncation=True,
            padding='max_length',
            max_length=self.max_seq_length,
            return_tensors='np'
        )
        # return all three for compatibility
        input_ids      = enc['input_ids']
        attention_mask = enc['attention_mask']
        # token_type_ids is present and is all zeros for single-sentence inputs
        token_type_ids = enc.get('token_type_ids', tf.zeros_like(input_ids))
        return input_ids, attention_mask, token_type_ids
        

    def build_model(self):
        ids  = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='input_ids')
        mask = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='attention_mask')
        ttid = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='token_type_ids')

        bert_encoder = TFBertModel.from_pretrained('bert-base-uncased')


        # 👉 declare the hidden-state size here (768 for bert-base)
        hidden_size = 768

        # ✨ FIX: pass inputs as a list, not kwargs

        cls_token = tf.keras.layers.Lambda(
            lambda x: bert_encoder(
                input_ids=x[0], attention_mask=x[1], token_type_ids=x[2]
                ).last_hidden_state[:, 0, :],
                output_shape=(hidden_size,)          # tell Keras the shape
                )([ids, mask, ttid])


        # ── 4. Classification head ────────────────────────────
        x   = tf.keras.layers.Dropout(0.2)(cls_token)
        out = Dense(1, activation='sigmoid')(x)

        model = Model([ids, mask, ttid], out)
        model.compile(optimizer=tf.keras.optimizers.Adam(self.lr),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        return model



In [7]:
clf = BertTweetClassifier(epochs=1)
model = clf.build_model()          # should build without the NotImplementedError


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [8]:
sample_texts = ["Massive fire downtown", "I love sunny days"]  # 2 dummy tweets
ids, mask, ttid = clf.encode(pd.Series(sample_texts))
print(model.predict([ids, mask, ttid]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
[[0.4317377 ]
 [0.62810254]]


In [9]:
# --- Encode tweets -------------------------------------------------
ids,  masks,  ttids  = clf.encode(df_train['text_cleaned'])
labels               = df_train['target'].values

print("Encoded   :", ids.shape, masks.shape, ttids.shape)
print("Labels    :", labels.shape, "positive =", labels.sum())


Encoded   : (7613, 160) (7613, 160) (7613, 160)
Labels    : (7613,) positive = 3271


In [11]:
from sklearn.model_selection import train_test_split

X_ids_tr,  X_ids_val, \
X_mask_tr, X_mask_val, \
X_tid_tr,  X_tid_val, \
y_tr,      y_val = train_test_split(
    ids, masks, ttids, labels,
    test_size=0.10, stratify=labels, random_state=42
)

model = clf.build_model()

history = model.fit(
    [X_ids_tr,  X_mask_tr,  X_tid_tr],  y_tr,
    validation_data=([X_ids_val, X_mask_val, X_tid_val], y_val),
    epochs=1,            # bump to 4-5 later if loss is still dropping
    batch_size=16,       # drop to 8 if you hit OOM
    verbose=1
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

429/429 ━━━━━━━━━━━━━━━━━━━━ 957s 2s/step - accuracy: 0.4364 - loss: 0.7599 - val_accuracy: 0.4278 - val_loss: 0.7308
